In [1]:
from csoundengine import *

e = Engine()
s = e.session()

using libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [2]:
tab = s.readSoundfile('../snd/piano-pitchtest-Cmaj.flac')
s.defInstr('test0', r'''
kfreq = p5
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')
s.sched('test0', 0, 2, kfreq=440)
ev = s.playSample(tab)

In [3]:
s.defInstr('test1', r'''
|kfreq|
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test1, kfreq=0)

In [4]:
s.defInstr('test', r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01)) * linsegr:a(0, 0.1, 1, 0.1, 0)
''')


Instr(test, ibus:5)

In [7]:
bus = s.assignBus(value=2000)
synth = s.sched('test', ibus=bus.token)
bus.set(260, 0.001)
synth

Button(description='Stop', style=ButtonStyle())

Output()

Synth(‖ test=403.0004 start=32.921 dur=-1 ibus=2)

In [8]:
bus.set(700)

In [9]:
buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test', dur=2, ibus=bus)
    e.writeBus(bus, 500, 0.001)
    

Unknown sync token: 1999


In [10]:
buses = [s.assignBus(value=600) for _ in range(200)]
s.engine.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(500, delay=0.001)
    

In [11]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig vco2 0.1, kfreq
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [12]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
e.writeBus(freqbus.token, 400, delay=1)
chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,p4,ibus,ifreqbus
404.0002 ‖,0.016,-1.000,0,403,404
p1,start,dur,p4,ibus,kgain
405.0002 ‖,0.028,-1.000,0,403,0.5


In [13]:
e.writeBus(freqbus.token, 800)
freqbus.set(500)
